In [1]:
import torchaudio
import os
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.autonotebook import tqdm

/tmp/ipykernel_3453639/3703226227.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [8]:
import librosa
from IPython.display import Audio, display

def check_audio_files(audio_path, resample_rate=6000):
    data, sr = librosa.load(audio_path)
    resample_data = librosa.resample(data, orig_sr=sr, target_sr=resample_rate)
    
    print(data.shape, resample_data.shape)
    display(Audio(data=data, rate=sr))
    display(Audio(data=resample_data, rate=resample_rate))

In [9]:
check_audio_files("/home/desild/work/academic/sem3/TrustworthyML-assignment/data/raw/vctk/VCTK-Corpus/VCTK-Corpus/wav48/p256/p256_290.wav", resample_rate=2000)

(136461,) (12378,)


In [ ]:
with open('/home/desild/work/academic/sem3/TrustworthyML-assignment/data/raw/vctk/VCTK-Corpus/VCTK-Corpus/speaker-info.txt', 'r') as f:
    speaker_info = f.readlines()
    columns = speaker_info[0].strip().split('\t')
    data = [line.strip().split('    ') for line in speaker_info[1:]]
    data = [[*d[0].split("  "), *d[1:]] for d in data]
    metadata = pd.DataFrame(data, columns=columns[0].split("  "))

In [ ]:
_temp = []
for idx, row in tqdm(metadata.iterrows(), total=metadata.shape[0]):
    speaker_id = row['ID']
    speaker_folder = f"/home/desild/work/academic/sem3/TrustworthyML-assignment/data/raw/vctk/VCTK-Corpus/VCTK-Corpus/wav48/p{speaker_id}"
    for file in os.listdir(speaker_folder):
        if file.endswith('.wav'):
            _temp.append({
                'filename': file,
                'speaker_id': speaker_id,
                'wav_path': os.path.join(speaker_folder, file),
                'text_path': os.path.join(speaker_folder, file.replace('wav48', 'txt').replace('.wav', '.txt')),
                **row.to_dict()
            })
            
metadata = pd.DataFrame(_temp)

  0%|          | 0/108 [00:00<?, ?it/s]

In [4]:
metadata = metadata.loc[~metadata["filename"].isnull()]

In [5]:
metadata

,filename,speaker_id,wav_path,text_path,ID,AGE,GENDER,ACCENTS,REGION
0,p225_193.wav,225,/home/desild/work/academic/sem3/TrustworthyML-...,/home/desild/work/academic/sem3/TrustworthyML-...,225,23,F,English,Southern England
1,p225_169.wav,225,/home/desild/work/academic/sem3/TrustworthyML-...,/home/desild/work/academic/sem3/TrustworthyML-...,225,23,F,English,Southern England
2,p225_073.wav,225,/home/desild/work/academic/sem3/TrustworthyML-...,/home/desild/work/academic/sem3/TrustworthyML-...,225,23,F,English,Southern England
3,p225_056.wav,225,/home/desild/work/academic/sem3/TrustworthyML-...,/home/desild/work/academic/sem3/TrustworthyML-...,225,23,F,English,Southern England
4,p225_310.wav,225,/home/desild/work/academic/sem3/TrustworthyML-...,/home/desild/work/academic/sem3/TrustworthyML-...,225,23,F,English,Southern England
...,...,...,...,...,...,...,...,...,...
43827,p376_098.wav,376,/home/desild/work/academic/sem3/TrustworthyML-...,/home/desild/work/academic/sem3/TrustworthyML-...,376,22,M,Indian,None
43828,p376_222.wav,376,/home/desild/work/academic/sem3/TrustworthyML-...,/home/desild/work/academic/sem3/TrustworthyML-...,376,22,M,Indian,None
43829,p376_095.wav,376,/home/desild/work/academic/sem3/TrustworthyML-...,/home/desild/work/academic/sem3/TrustworthyML-...,376,22,M,Indian,None
43830,p376_067.wav,376,/home/desild/work/academic/sem3/TrustworthyML-...,/home/desild/work/academic/sem3/TrustworthyML-...,376,22,M,Indian,None


In [6]:
metadata["AGE"].value_counts()

AGE
23    11837
22    10525
21     4391
19     3769
26     2915
24     2457
20     1970
18     1859
25     1181
29      882
28      424
33      423
32      422
38      389
27      388
Name: count, dtype: int64

In [7]:
import librosa

def dur(x):
    librosa_audio, sr = librosa.load(x, sr=None)
    return librosa_audio.shape[0] / sr

def age_group(age):
    age = int(age)
    if age < 23:
        return "<20"
    elif age < 28:
        return "20-28"
    else:
        return "28<"
    
def accent_group(accent):
    return accent.split(" ")[0]

metadata["duration"] = metadata.wav_path.apply(dur)
metadata["AGEg"] = metadata["AGE"].apply(age_group)
metadata["ACCENTS"] = metadata["ACCENTS"].apply(accent_group)

In [8]:
list_obj = metadata.groupby(["AGEg", "GENDER", "speaker_id", "ACCENTS"]).agg({
    "filename": list,
    "duration": "sum"
})

In [9]:
list_obj = list_obj.reset_index()
list_obj["num_files"] = list_obj["filename"].apply(len)

In [10]:
list_obj

,AGEg,GENDER,speaker_id,ACCENTS,filename,duration,num_files
0,20-28,F,225,English,"[p225_193.wav, p225_169.wav, p225_073.wav, p22...",988.155146,231
1,20-28,F,229,English,"[p229_105.wav, p229_326.wav, p229_360.wav, p22...",1212.982083,379
2,20-28,F,231,English,"[p231_345.wav, p231_444.wav, p231_012.wav, p23...",1200.353542,456
3,20-28,F,233,English,"[p233_038.wav, p233_061.wav, p233_377.wav, p23...",1353.617771,372
4,20-28,F,236,English,"[p236_384.wav, p236_241.wav, p236_392.wav, p23...",1342.153021,492
...,...,...,...,...,...,...,...
103,<20,M,334,American,"[p334_075.wav, p334_406.wav, p334_216.wav, p33...",1355.014208,423
104,<20,M,345,American,"[p345_086.wav, p345_150.wav, p345_339.wav, p34...",1335.757938,397
105,<20,M,360,American,"[p360_017.wav, p360_391.wav, p360_246.wav, p36...",1285.264021,424
106,<20,M,363,Canadian,"[p363_111.wav, p363_223.wav, p363_074.wav, p36...",1171.361417,419


In [11]:
list_obj["count"] = list_obj["filename"].apply(len)
list_obj = list_obj.sort_values(by="count", ascending=False)

stats = list_obj[["AGEg", "GENDER", "speaker_id", "ACCENTS", "duration", "count"]]

In [12]:
stats.to_csv("vctk_speaker_stats.csv", index=False)

In [13]:
stats

,AGEg,GENDER,speaker_id,ACCENTS,duration,count
56,<20,F,239,English,1540.950354,503
4,20-28,F,236,English,1342.153021,492
9,20-28,F,264,Scottish,1800.138312,483
35,20-28,M,259,English,1713.879062,481
60,<20,F,250,English,1346.546646,481
...,...,...,...,...,...,...
98,<20,M,302,Canadian,1289.118250,311
45,20-28,M,364,Irish,991.359646,301
107,<20,M,376,Indian,1403.065167,292
0,20-28,F,225,English,988.155146,231


In [14]:
list_obj["GENDER"].unique(), list_obj["AGEg"].unique(), list_obj["ACCENTS"].unique()

(array(['F', 'M'], dtype=object),
 array(['<20', '20-28', '28<'], dtype=object),
 array(['English', 'Scottish', 'NorthernIrish', 'Irish', 'American',
        'SouthAfrican', 'Australian', 'Canadian', 'NewZealand', 'Indian',
        'Welsh'], dtype=object))

In [15]:
speaker_id = set()
unseen_id = set()
_AVL = set()

for s in list_obj["GENDER"].unique():
    for a in list_obj["AGEg"].unique():
        for acc in list_obj["ACCENTS"].unique():
            subset = list_obj[(list_obj["GENDER"] == s) & (list_obj["AGEg"] == a) & (list_obj["ACCENTS"] == acc)]
            subset = subset.sort_values(by="duration", ascending=False)
            
            try:
                speaker_id.add(subset.iloc[0, 2])
                unseen_id.update(subset.iloc[1:, 2].tolist())
                _AVL.add((s, a, acc))
            except IndexError:
                print(f"No speaker for {s}, {a}, {acc}")
                

No speaker for F, <20, Australian
No speaker for F, <20, NewZealand
No speaker for F, <20, Indian
No speaker for F, 20-28, Australian
No speaker for F, 20-28, Welsh
No speaker for F, 28<, English
No speaker for F, 28<, Scottish
No speaker for F, 28<, NorthernIrish
No speaker for F, 28<, Irish
No speaker for F, 28<, SouthAfrican
No speaker for F, 28<, Australian
No speaker for F, 28<, Canadian
No speaker for F, 28<, NewZealand
No speaker for F, 28<, Indian
No speaker for F, 28<, Welsh
No speaker for M, <20, SouthAfrican
No speaker for M, <20, Australian
No speaker for M, <20, NewZealand
No speaker for M, <20, Welsh
No speaker for M, 20-28, American
No speaker for M, 20-28, Canadian
No speaker for M, 20-28, NewZealand
No speaker for M, 20-28, Welsh
No speaker for M, 28<, NorthernIrish
No speaker for M, 28<, Irish
No speaker for M, 28<, American
No speaker for M, 28<, SouthAfrican
No speaker for M, 28<, Canadian
No speaker for M, 28<, NewZealand
No speaker for M, 28<, Indian
No speaker fo

In [16]:
pd.Series([x[0] for x in _AVL]).value_counts(), pd.Series([x[1] for x in _AVL]).value_counts(), pd.Series([x[2] for x in _AVL]).value_counts()

(F    18
 M    17
 Name: count, dtype: int64,
 20-28    16
 <20      15
 28<       4
 Name: count, dtype: int64,
 English          5
 Scottish         5
 American         4
 Irish            4
 NorthernIrish    4
 Canadian         3
 Indian           3
 SouthAfrican     3
 Australian       2
 NewZealand       1
 Welsh            1
 Name: count, dtype: int64)

In [17]:
full_data = metadata[metadata["speaker_id"].isin(speaker_id)].reset_index(drop=True)

In [18]:
full_data["unq_id"] = full_data["speaker_id"] + "_" + full_data["filename"].str.replace(".WAV.wav", "")

In [19]:
speaker_file = full_data.groupby("speaker_id")['unq_id'].apply(list).reset_index()


In [20]:
speaker_id

{'227',
 '244',
 '245',
 '248',
 '249',
 '251',
 '252',
 '253',
 '256',
 '261',
 '264',
 '268',
 '274',
 '275',
 '281',
 '288',
 '292',
 '293',
 '294',
 '295',
 '298',
 '301',
 '304',
 '306',
 '307',
 '311',
 '312',
 '314',
 '316',
 '323',
 '326',
 '335',
 '347',
 '374',
 '376'}

In [21]:
import random

train_ = []
val_ = []
for speaker in speaker_file.to_dict(orient='records'):
    speaker_id = speaker['speaker_id']
    filenames = speaker['unq_id']
    _t = random.sample(filenames, k=int(len(filenames)*0.8))
    train_.extend(_t)
    val_.extend([f for f in filenames if f not in _t])

In [22]:
len(train_), len(val_)

(11320, 2848)

In [23]:
val_

['227_p227_099.wav',
 '227_p227_262.wav',
 '227_p227_142.wav',
 '227_p227_219.wav',
 '227_p227_247.wav',
 '227_p227_281.wav',
 '227_p227_136.wav',
 '227_p227_338.wav',
 '227_p227_018.wav',
 '227_p227_029.wav',
 '227_p227_194.wav',
 '227_p227_258.wav',
 '227_p227_196.wav',
 '227_p227_058.wav',
 '227_p227_316.wav',
 '227_p227_153.wav',
 '227_p227_203.wav',
 '227_p227_322.wav',
 '227_p227_124.wav',
 '227_p227_184.wav',
 '227_p227_261.wav',
 '227_p227_216.wav',
 '227_p227_356.wav',
 '227_p227_257.wav',
 '227_p227_034.wav',
 '227_p227_380.wav',
 '227_p227_104.wav',
 '227_p227_025.wav',
 '227_p227_175.wav',
 '227_p227_195.wav',
 '227_p227_390.wav',
 '227_p227_170.wav',
 '227_p227_313.wav',
 '227_p227_342.wav',
 '227_p227_209.wav',
 '227_p227_003.wav',
 '227_p227_222.wav',
 '227_p227_382.wav',
 '227_p227_348.wav',
 '227_p227_337.wav',
 '227_p227_039.wav',
 '227_p227_383.wav',
 '227_p227_162.wav',
 '227_p227_392.wav',
 '227_p227_127.wav',
 '227_p227_391.wav',
 '227_p227_100.wav',
 '227_p227_03

In [24]:
import random

val_data = full_data[full_data['unq_id'].isin(val_)].reset_index(drop=True)
train_data = full_data[full_data['unq_id'].isin(train_)].reset_index(drop=True)
unseen_data = metadata[metadata["speaker_id"].isin(unseen_id)].reset_index(drop=True)

In [25]:
val_data.shape, train_data.shape, unseen_data.shape

((2848, 12), (11320, 12), (29664, 11))

In [26]:
train_data.to_csv("/home/desild/work/academic/sem3/TrustworthyML-assignment/data/raw/vctk/train_data_top.csv", index=False)
val_data.to_csv("/home/desild/work/academic/sem3/TrustworthyML-assignment/data/raw/vctk/val_data_top.csv", index=False)
unseen_data.to_csv("/home/desild/work/academic/sem3/TrustworthyML-assignment/data/raw/vctk/unseen_data.csv", index=False)

In [27]:
set1 = set(unseen_data["speaker_id"])
set2 = set(train_data["speaker_id"])

In [28]:
set1.intersection(set2)

set()